In [ ]:
# DNN_A with 1025 input and 2050 output
# DNN_C with 2050 input and 2050 output
# Half of the data is used to train DNN_A and the other half is given to trained DNN_A to generate data for DNN_C
# DNN_C has it's own cost function(customized)

In [1]:
#import libraries.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
#read necessary files, contarins mixed and clean features.
def data_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

In [4]:
def phase_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed_phase" +str(i) + ".p", "rb")
    angle = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    return angle

In [5]:
h_A = [1025,1025,1025]
h_C = [4100, 4100, 4100]

In [5]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

In [27]:
I=0
def data_gen(batch_size, data_len):
    global I
    while True:             #this line is just because keras needs infinite generators
        for I in range(0,data_len-batch_size,batch_size): 
            h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
            ftr = h5f['mixed0'][:,I:I+batch_size]
            h5f.close()
            h5f = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
            temp = h5f['clean0'][:,I:I+batch_size]
            target = temp
            h5f.close()
            yield(ftr.T, target.T)

In [13]:
# X, y = data_reading(0)
X_dim= 1025
y_dim = 2050
estimator_A = Sequential()
estimator_A.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(y_dim, kernel_initializer='normal'))
# Compile model
estimator_A.compile(loss='mean_squared_error', optimizer='adam')

In [9]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

# history = LossHistory()

In [25]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
ftr_shape = h5f['mixed0'].shape
h5f.close()
batch_size = 128
steps_per_epoch = int(ftr_shape[1]/(2*batch_size))

In [21]:
h5f = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
clean_shape = h5f['clean0'][:,0:128]
h5f.close()

In [23]:
clean_shape.shape

(2050, 128)

In [15]:
ftr_shape

(1025, 939485)

In [18]:
steps_per_epoch

3669

In [12]:
# gg = data_gen(batch_size, int(ftr_shape[0]/2))

In [48]:
# print(next(gg)[0].shape)

(128, 1025)


In [28]:
tic = time.time()
estimator_A.fit_generator(generator=data_gen(batch_size, int(ftr_shape[0]/2)),
                          steps_per_epoch=steps_per_epoch, 
                          epochs=50,
                          shuffle=True,
                          verbose=1)
toc  =time.time()

Epoch 1/50
3669/3669 [==============================] - 105s 29ms/step - loss: 0.0055
Epoch 2/50
3669/3669 [==============================] - 96s 26ms/step - loss: 3.2683e-04
Epoch 3/50
3669/3669 [==============================] - 95s 26ms/step - loss: 1.3839e-04
Epoch 4/50
3669/3669 [==============================] - 95s 26ms/step - loss: 7.5481e-05
Epoch 5/50
3669/3669 [==============================] - 95s 26ms/step - loss: 5.5282e-05
Epoch 6/50
3669/3669 [==============================] - 95s 26ms/step - loss: 6.3811e-05
Epoch 7/50
3669/3669 [==============================] - 95s 26ms/step - loss: 3.2245e-05
Epoch 8/50
3669/3669 [==============================] - 95s 26ms/step - loss: 2.9843e-05
Epoch 9/50
3669/3669 [==============================] - 95s 26ms/step - loss: 2.1974e-05
Epoch 10/50
3669/3669 [==============================] - 95s 26ms/step - loss: 2.0948e-05
Epoch 11/50
3669/3669 [==============================] - 95s 26ms/step - loss: 1.9141e-05 1s 
Epoch 12/50
3669/3

In [29]:
print('time_of_train =', (toc - tic)/60)

time_of_train = 79.79761300086975


In [30]:
estimator_A.save('Models/Two_stage/trained_model.h5')

In [31]:
estimator_A.save_weights('Models/Two_stage/weights.h5')

In [6]:
estimator_A = load_model('Models/Two_stage/trained_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [17]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [258]:
test_point_start0 = 1000
test_point_stop0 = 1400

test_point_start = 469750
test_point_stop = 469850


In [259]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
test_input = h5f['mixed0'][:,test_point_start0 : test_point_stop0]
h5f.close()

In [260]:
h5f = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
test_target = h5f['clean0'][:, test_point_start0 : test_point_stop0]
h5f.close()

In [261]:
h5f = h5py.File('TIMIT/Organized/concatenated/phase0.hdf5','r')
angle = h5f['phase0'][:,test_point_start0 : test_point_stop0]
h5f.close()

In [262]:
print(test_input.shape)
print(test_target.shape)
print(angle.shape)

(1025, 400)
(2050, 400)
(1025, 400)


In [263]:
prediction = estimator_A.predict(test_input.T)

In [264]:
prediction.shape

(400, 2050)

In [265]:
s1 = reconstruct(prediction[:,1025:2050], angle.T)

In [266]:
clean_recon = reconstruct(test_target[1025:2050,:].T, angle.T)

In [267]:
# mixed_recon = reconstruct(test_input[0:1025,:].T, angle.T)

In [268]:
sd.play(s1*10, 16000)

In [255]:
sd.play(mixed_recon*10, 16000)

In [246]:
a.shape

(1025, 300)

In [247]:
b.shape

(300, 1025)

In [248]:
st_recon = stoi(s1, clean_recon, 16000, extended=False)

In [249]:
st_recon

0.8282933397259574

In [250]:
st_mixed = stoi(mixed_recon, clean_recon, 16000, extended=False)

In [251]:
st_mixed

0.6989585388076244

In [252]:
sdr_recon, sir_recon, sar_recon, perm_recon = separation.bss_eval_sources(s1, clean_recon, compute_permutation=True)

In [253]:
sdr_mixed, sir_mixed, sar_mixed, perm_mixed = separation.bss_eval_sources(mixed_recon, clean_recon, compute_permutation=True)

In [254]:
print('SDR_recon=', sdr_recon)
print('SDR_mixed=', sdr_mixed)

SDR_recon= [16.44320281]
SDR_mixed= [7.82182918]


In [ ]:
1025:2050